In [1]:
import mlflow

In [2]:
mlflow.log_param('param_a', 5)
mlflow.log_metric('metric_b', 167)

In [3]:
mlflow.end_run()

In [4]:
with mlflow.start_run():
    mlflow.log_param('param_a', 1)
    mlflow.log_metric('metric_b', 2)
    mlflow.log_metric('metric_b', 4)
    mlflow.log_metric('metric_b', 5)

In [6]:
mlflow.set_experiment("my-experiment")

In [ ]:
# Load the Data
data = pd.read_csv('Iris.csv')

X = data.drop(['Id', 'Species'], axis=1)
y = data['Species']

# Train Test split
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.4, random_state=5)

In [7]:
import pandas as pd
import os

import mlflow
from mlflow import log_metric, log_param, log_artifacts
from mlflow.sklearn import log_model

from sklearn import metrics
from sklearn.ensemble import RandomForestClassifier
from sklearn.model_selection import train_test_split

In [14]:
k=12

In [9]:
with mlflow.start_run():
    # Train model
    rfc = RandomForestClassifier(n_estimators=50)
    rfc.fit(X_train, y_train)

    # Accuracy score
    y_pred = rfc.predict(X_test)
    accuracy = metrics.accuracy_score(y_test, y_pred)
    print("Accuracy score: %.2f"% accuracy)
    
    # Track the run
    log_param("n_estimators", rfc.n_estimators)
    log_metric("accuracy", accuracy)
    log_model(rfc, "rfc_model")

    # Log an artifact (output file)
    if not os.path.exists("outputs"):
        os.makedirs("outputs")
    with open("outputs/test.txt", "w") as f:
        f.write("hello world!")
    log_artifacts("outputs")
    
    print("Model saved in run %s" % mlflow.active_run().info.run_uuid)

Accuracy score: 0.95
Model saved in run 00dc15c83fca491e95410fa5a4169f24


In [10]:
from mlflow.models.signature import infer_signature

with mlflow.start_run():
    mlflow.log_param("n_estimators", rfc.n_estimators)
    mlflow.log_metric("accuracy", accuracy)
    
    signature = infer_signature(X_train, rfc.predict(X_train))
        
    input_example = {
      "SepalLengthCm": 5.1,
      "SepalWidthCm": 3.5,
      "PetalLengthCm": 1.4,
      "PetalWidthCm": 0.2
    }
    mlflow.sklearn.log_model(rfc, "rfc_model", 
                             input_example=input_example,
                             signature=signature)


    # Log an artifact (output file)
    if not os.path.exists("outputs"):
        os.makedirs("outputs")
    with open("outputs/test.txt", "w") as f:
        f.write("hello world!")
    log_artifacts("outputs")
    
    print("Model saved in run %s" % mlflow.active_run().info.run_uuid)



Model saved in run f0bd591fa0834949bcfeb59717e507eb


run this in python script in the same directory: !mlflow ui

In [13]:
mlflow.set_tracking_uri("http://kubernetes.docker.internal:5000")
mlflow.set_experiment("my-experiment")

INFO: 'my-experiment' does not exist. Creating a new experiment
